### Topic2wordvector using tensorflow 

jiwan, Seo 

#### Abstract 
 Word2vec은 자연어를 분석하고 활용하기 위해 널리 사용하는 방법 중 하나이다. 이 방법은 간단한 인공 신경망 학습을 이용하여 학습 문서의 단어를 벡터 공간에 사영한다. 하지만, 기존의 방법에서는 단어는 각 하나의 벡터 공간에 사영되고, 이로 인해 동음이의어나 주제에 따른 단어 할당에 어려움이 있다. 본 글에서는 토픽 모델링으로 알려진 LDA(Latent Dirichlet Allocation) 방법과 Word2Vec을 활용하여 단어를 주제별로 학습하는 모델을 구축한다. 새로운 문서가 들어오면 기 학습한 LDA 모형을 기반으로 해당 문서의 토픽 정보를 추론할 수 있으며, 토픽 정보를 활용하여 해당 문서의 단어 벡터들을 추출할 수 있다. 본 방법은 일반적으로 다양한 주제가 혼합되어 있는 환경에서 토픽별로 단어 벡터를 추론할 수 있다는 장점이 있다.   
***keyword***: LDA, Word2vec 
 
 

#### 1. Introdcution 

 Word2vec[?]은 자연어를 분석하고 활용하기 위해 널리 사용되는 방법이다. 이 방법은 두 단계의 인공 신경망을 거쳐 학습 문서로부터 단어를 특정 벡터 공간에 사영하는 방법이다. 학습 과정에서 비슷한 문맥으로 사용된 단어는 점점 비슷한 공간에 위치하게 되는데, 이러한 단어 벡터 정보는 단어, 문서의 직접적인 비교 뿐 아니라 챗봇의 RNN이나 LSTM 등 인공 신경망 학습의 입력 값으로도 활용된다. word2vec은 단어 사전 (Vocab size) X 벡터 공간(embedding size) 크기의 행렬을 업데이트하는데, 해당 행렬은 각 단어의 벡터 정보로 활용된다. 결과적으로 문서에서 사용된 모든 단어는 각 하나의 벡터 정보를 가지게 된다. 
 
 하지만, 실제로 단어는 의미나 쓰임, 문서의 주제에 따라 다르게 활용되기도 한다. 예를 들어, 수학 관련 문서에서 '정의'와 정치 관련 문서에서 '정의'는 서로 다른 의미로 사용된다. 기존의 Word2Vec에서는 모든 단어는 하나의 벡터 정보를 가지기 때문에 이러한 문제에 한계를 보인다. 위의 예제에서 정의는 수학 관련 용어들과 정치 관련 용어들의 중간쯤에 위치하게 된다. 이러한 문제를 해결하기 위해 서로 다른 학습 데이터를 활용할 수도 있지만, 이 경우, 두 모델 사이의 단어 사전의 크기가 달라져 활용하는데 어려움이 있을 수 있다. 
 
 본 글에서는 tensorflow와 gensim 라이브러리를 활용하여 학습 문서에 내제되어 있는 주제를 바탕으로 단어를 벡터 공간에 사영하는 방법을 구현한다. 본 방법은 일반적으로 다양한 주제가 혼합되어 있는 문서 환경에서 효과적으로 단어 벡터를 추론할 수 있다는 장점이 있다. 새로운 문서가 등장해도 LDA 모형과 word2vec 모형을 활용하여 다양한 주제 관점에서 유사성을 비교할 수 있다. 
 
 

- 기존의 방법은 문맥적인 활용이 두 가지 이상(즉, 동음이의어)를 분별할 수 없다는 점과 
- 도메인에 ㄷ

 
 
- 정의란 무엇인가? 이는 사람이 가지고 있는 지식에 따라 다른 대답이 나올 수 있다. 이는 정의가 동음이의어의 성격을 가지고 있기 때문이다. 
    - 기본적으로 정의란 justice의 의미로 가장 많이 사용된다. 하지만 이과생들에게는 definition 의미로 정의라는 단어를 많이 사용한다
    - 기존의 word2vec에서는 학습 데이터에 따라 정의가 justice나 defination의 중간 어딘가에 매칭될 것이다
    - 이러한 단어가 많아질 수록, 벡터 공간의 일부분을 동음이의어가 차지하는 결과를 보일 것이며, 이러한 문제를 해결하기 위해서는 더 많은 차원이 필요해 질 것이다. 

- 따라서, 정의라는 단어는 주제적인 측면에서 1차적으로 분류되고, 그 다음 의미적인 분석을 수행해야 한다 
    - 그래서 아이디어는 LDA를 활용하여 1차적으로 k개의 잠재적 토픽으로 분류하고, 이를 바탕으로 k X W x V의 Word2vec 학습을 수행한다 
    - 즉, one-hot encoding 을 통해 합쳐질 때, k의 가중치를 반영하여 여러 w x v 메트릭스를 동시에 학습하는 것이다. 
    
- 우선 구현은 다른 라이브러리를 적극적으로 활용하느 것으로 하되, 여의치 않으면 작은 사이즈로 CBOW만 구현 해보는 것으로 하자  

---

- 데이터 전처리 -> LDA(Vocab의 사이즈는 LDA와 W2V이랑 동일) -> k(topic size) X W(vocab size) X vector(word vector size)   

---
  
참조 사이트 
- https://hulk89.github.io/neural%20machine%20translation/2017/05/08/Word2Vec-impl/ 
- http://solarisailab.com/archives/374

In [385]:
#library definition 
import collections 
import math 
import os 
import zipfile 

import numpy as np 
import tensorflow as tf 

import json
import pandas as pd

import random
import gensim

num_topics = 5

#### step 1: 텍스트 데이터 읽어오기 


In [386]:
# data read 
def read_Amazon_review_data(file_path):
    review_list = []
    with open (file_path) as json_file:
        d = json_file.read()

    split_data = d.split("\n")
    for text in split_data:
        if text.strip():
            text_list = json.loads(text)
            review_list.append({"ID": text_list["reviewerID"], 
                                "Review": text_list["reviewText"], 
                                "Rating": text_list["overall"], 
                                "asin": text_list["asin"],
                                "timestemp":text_list["unixReviewTime"]
                                         
                               })

    return review_list

In [387]:
read_data = read_Amazon_review_data("reviews_Musical_Instruments_5.json")

In [388]:
read_data[:2]

[{'ID': 'A2IBPI20UZIR0U',
  'Review': "Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,",
  'Rating': 5.0,
  'asin': '1384719342',
  'timestemp': 1393545600},
 {'ID': 'A14VAT5EAX3D9S',
  'Review': "The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the screens carries a small hint of the smell of an old grape candy I used to buy, so for reminiscent's sake, I cannot stop putting the pop filter next to my nose and smelling it after recording. :DIf you needed a pop filter, this will work just as well as the expensive ones, and it may even come with a pleasing aroma like mine did!Buy this product! :]",
  'Rating': 5.0,
  'asin': '1384719

In [389]:
data_word = [] 
for line in read_data: 
    line["Review"] = list(gensim.utils.simple_preprocess(str(line["Review"]), deacc=True))
    data_word.append(line["Review"])

#### step 2: LDA 수행하기 

In [390]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


bigram = gensim.models.Phrases(data_word, min_count = 5, threshold = 100)
bigram_mode = gensim.models.phrases.Phraser(bigram)

def remove_stopwords(texts): 
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mode[doc] for doc in texts]

data_words_nostops = remove_stopwords(data_word)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


# Create Dictionary
id2word = gensim.corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [391]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
id2word=id2word,
num_topics=num_topics,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto',
per_word_topics=True)

In [392]:
lda_model.print_topics()

[(0,
  '0.018*"stand" + 0.015*"case" + 0.014*"think" + 0.013*"pick" + 0.012*"play" + 0.012*"picks" + 0.011*"strap" + 0.010*"simple" + 0.010*"size" + 0.010*"fit"'),
 (1,
  '0.016*"recording" + 0.015*"mic" + 0.010*"music" + 0.010*"unit" + 0.009*"tube" + 0.009*"record" + 0.008*"usb" + 0.008*"yet" + 0.008*"microphone" + 0.008*"ipad"'),
 (2,
  '0.037*"pedal" + 0.030*"amp" + 0.030*"tone" + 0.018*"sound" + 0.014*"pedals" + 0.014*"sounds" + 0.013*"play" + 0.009*"volume" + 0.008*"clean" + 0.008*"gain"'),
 (3,
  '0.020*"one" + 0.018*"strings" + 0.017*"great" + 0.017*"like" + 0.016*"guitar" + 0.015*"good" + 0.013*"well" + 0.011*"get" + 0.010*"really" + 0.010*"sound"'),
 (4,
  '0.030*"tuner" + 0.025*"easy" + 0.024*"string" + 0.024*"capo" + 0.023*"guitar" + 0.021*"tune" + 0.018*"guitars" + 0.016*"acoustic" + 0.013*"tuning" + 0.013*"instrument"')]

#### step 3: word2vec 구현하기 using tensorflow 

In [393]:
len(texts)

10261

In [394]:
test_texts = texts[:int(len(texts)/10)]

In [395]:
def generate_input(batch_size, dataset,id2word, window_size):    
    random.shuffle(dataset)
    data = []
    label = []
    for doc in dataset : 
        if len(doc) > window_size:
            for idx in range(int(window_size/2), len(doc)-(window_size - int(window_size/2))):
                front = idx - int(window_size/2)
                rear = idx + (window_size - int(window_size/2))
                #flatten.append({'data': doc[front:idx] + doc[idx:rear], 'label':doc[idx]})
                data.append(id2word.doc2idx(doc[front:idx] + doc[idx+1:rear])) 
                label.append(id2word.doc2idx([doc[idx]]))
    
    n_batch = len(data)//batch_size
    data = data[:n_batch * batch_size]
    label = label[:n_batch * batch_size]
    
    #return (data, label)
    for ii in range(0, len(data), batch_size):
        yield n_batch, data[ii:ii+batch_size], label[ii:ii+batch_size]
            

In [396]:
vocab_size = len(id2word)
embedding_size = 10
window_size = 5
batch_size = 20
#input_ , label = generate_input(batch_size, texts, id2word, window_size)
num_sampled = batch_size#vocab_size//2

In [397]:
embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))

nce_weights = tf.Variable(tf.truncated_normal([vocab_size, embedding_size], stddev = 1.0/ math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocab_size]))

train_inputs = tf.placeholder(tf.int32, shape=[batch_size, window_size-1])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

In [398]:
embed = [tf.reduce_mean(tf.nn.embedding_lookup(embeddings, train_inputs[idx]), axis=0)
         for idx in range(batch_size)] # 토픽 분포의 값을 곱해준다 

loss = tf.reduce_mean(tf.nn.nce_loss(nce_weights,
                                     nce_biases,
                                     train_labels,
                                     embed,
                                     num_sampled,
                                     vocab_size))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

init = tf.global_variables_initializer()

In [399]:
with tf.Session() as sess:
    init.run()
    for i in range(2):
        for ii, (n_batch, x, y) in enumerate(generate_input(batch_size, texts, id2word, window_size)):
            feed_dict = {train_inputs: x, train_labels: y}
            #print(tf.convert_to_tensor(x).shape)
            _, loss_val = sess.run([optimizer, loss], feed_dict=feed_dict)
            #test_embed = sess.run([embed], feed_dict=feed_dict)
            #print(test_embed)
            if ii % n_batch == 0: 
                print(i, " epoch : ",ii,"/",n_batch," = ", loss_val)
    emb_weights = sess.run(embeddings)

nomal_weights = emb_weights
np.savetxt('dataset', emb_weights, fmt='%.5e', delimiter='\t')

98.031265
5.4575486
3.9435031
3.6200683
3.2777276


In [400]:
with open('meta', 'w') as f:
    for i , weight in enumerate(emb_weights):
        f.write("{}\t{}\n".format(id2word[i], weight))
        

#### step 4: LDA 수행 결과를 반영하여 word2Vec 수행하기 


In [401]:
def generate_input2(batch_size, dataset,id2word, window_size):    
    random.shuffle(dataset)
    data = []
    label = []
    for doc in dataset : 
        if len(doc) > window_size:
            for idx in range(int(window_size/2), len(doc)-(window_size - int(window_size/2))):
                front = idx - int(window_size/2)
                rear = idx + (window_size - int(window_size/2))
                #flatten.append({'data': doc[front:idx] + doc[idx:rear], 'label':doc[idx]})
                data.append(id2word.doc2idx(doc[front:idx] + doc[idx+1:rear])) 
                label.append(id2word.doc2idx([doc[idx]]))
    
    n_batch = len(data)//batch_size
    data = data[:n_batch * batch_size]
    label = label[:n_batch * batch_size]
    
    #return (data, label)
    for ii in range(0, len(data), batch_size):
        return data[ii:ii+batch_size], label[ii:ii+batch_size]

In [402]:
embeddings = tf.Variable(tf.random_uniform([num_topics, vocab_size, embedding_size], -1.0, 1.0))

nce_weights = tf.Variable(tf.truncated_normal([num_topics, vocab_size, embedding_size], 
                                              stddev = 1.0/ math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([num_topics, vocab_size]))

In [403]:
train_inputs = tf.placeholder(tf.int32, shape=[batch_size, window_size-1])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
topic_w = tf.placeholder(tf.float32, shape=[batch_size, num_topics])


embed = [topic_w[idx][topic]*tf.reduce_mean(tf.nn.embedding_lookup(
                                                embeddings[topic], train_inputs[idx]), axis=0)
         for idx in range(batch_size) for topic in range(num_topics) ] # 토픽 분포의 값을 곱해준다 


loss = tf.reduce_mean([tf.reduce_mean(tf.nn.nce_loss(nce_weights[idx],
                                     nce_biases[idx],
                                     train_labels,
                                     embed[idx*batch_size:idx*batch_size+batch_size],
                                     num_sampled,
                                     vocab_size)) for idx in range(num_topics)  ] )

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.8).minimize(loss)

init = tf.global_variables_initializer()

In [406]:
with tf.Session() as sess:
    init.run()
    for i in range(10):
        #for ii, (x, y) in enumerate(generate_input(batch_size, texts, id2word, window_size)):
        for ii, (n_batch, x, y) in enumerate(generate_input(batch_size,test_texts, id2word, window_size)):
            final_list = [] 
            for list_a in x: 
                temp_list = []
                for one_factor in list_a: 
                    temp_list += [(one_factor, 1)]
                final_list.append(temp_list)
                
            topic_w_ = []
            for list_ in final_list: 
                temp_val = []
                for (num, val) in lda_model[list_][0]: 
                    temp_val.append(val)
                topic_w_.append(temp_val)
            
            feed_dict = {train_inputs: x, train_labels: y, topic_w:topic_w_}
            #print(tf.convert_to_tensor(x).shape)
            _, loss_val = sess.run([optimizer, loss], feed_dict=feed_dict)
            #test_embed = sess.run([embed], feed_dict=feed_dict)
            if ii % int(n_batch*0.2) == 0: 
                print(i, " epoch : ",ii,"/",n_batch," = ", loss_val)
    emb_weights = sess.run(embeddings)
    


0  epoch :  0 / 1718  =  87.26283
0  epoch :  343 / 1718  =  82.10927
0  epoch :  686 / 1718  =  68.635025
0  epoch :  1029 / 1718  =  62.604473
0  epoch :  1372 / 1718  =  71.609634
0  epoch :  1715 / 1718  =  73.85561
1  epoch :  0 / 1718  =  61.195698
1  epoch :  343 / 1718  =  58.388176
1  epoch :  686 / 1718  =  62.90618
1  epoch :  1029 / 1718  =  59.698303
1  epoch :  1372 / 1718  =  52.951775
1  epoch :  1715 / 1718  =  59.68952
2  epoch :  0 / 1718  =  72.08372
2  epoch :  343 / 1718  =  54.89509
2  epoch :  686 / 1718  =  50.61388
2  epoch :  1029 / 1718  =  67.722694
2  epoch :  1372 / 1718  =  60.200745
2  epoch :  1715 / 1718  =  56.70807
3  epoch :  0 / 1718  =  55.693043
3  epoch :  343 / 1718  =  66.25566
3  epoch :  686 / 1718  =  49.972313
3  epoch :  1029 / 1718  =  47.38868
3  epoch :  1372 / 1718  =  34.910423
3  epoch :  1715 / 1718  =  43.96855
4  epoch :  0 / 1718  =  50.49537
4  epoch :  343 / 1718  =  45.058132
4  epoch :  686 / 1718  =  52.8167
4  epoch :  10

In [407]:
word_list = [id2word[idx] for idx in range(len(id2word))]

In [408]:
for idx in range(num_topics):    
    np.savetxt('dataset_topic'+str(idx), emb_weights[idx], fmt='%.5e', delimiter='\t')
    with open('meta_topic'+str(idx), 'w') as f:
        for i , weight in enumerate(emb_weights[idx]):
            f.write("{}\t{}\n".format(id2word[i], weight))

#### test 
- 일단 몇 개의 벡터를 서로 비교할 수 있게 

In [409]:
print(id2word.doc2bow(['buy'])[0][0] , id2word.doc2bow(['amazon'])[0][0])

1 0


In [418]:
from scipy import spatial

word1 = 'stand'
word2 = 'music'

el_1 = nomal_weights[id2word.doc2bow([word1])[0][0]]
el_2 = nomal_weights[id2word.doc2bow([word2])[0][0]]

result = 1 - spatial.distance.cosine(el_1, el_2)
print(word1, " ~ ", word2 , " : " ,result)

print("\n--------------------------\n")
for idx in range(num_topics):
    el_1 = emb_weights[idx][id2word.doc2bow([word1])[0][0]]
    el_2 = emb_weights[idx][id2word.doc2bow([word2])[0][0]]
    result = 1 - spatial.distance.cosine(el_1, el_2)
    print(lda_model.print_topic(idx, 5),  " \n", word1, " ~ ", word2 ," : ",result , "\n")


stand  ~  music  :  0.3746774196624756

--------------------------

0.018*"stand" + 0.015*"case" + 0.014*"think" + 0.013*"pick" + 0.012*"play"  
 stand  ~  music  :  -0.03944920003414154 

0.016*"recording" + 0.015*"mic" + 0.010*"music" + 0.010*"unit" + 0.009*"tube"  
 stand  ~  music  :  -0.4327864944934845 

0.037*"pedal" + 0.030*"amp" + 0.030*"tone" + 0.018*"sound" + 0.014*"pedals"  
 stand  ~  music  :  -0.8720686435699463 

0.020*"one" + 0.018*"strings" + 0.017*"great" + 0.017*"like" + 0.016*"guitar"  
 stand  ~  music  :  -0.11185695976018906 

0.030*"tuner" + 0.025*"easy" + 0.024*"string" + 0.024*"capo" + 0.023*"guitar"  
 stand  ~  music  :  0.3585844039916992 

